### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File 172 Word Twogram"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "The_Most_File_172_Word_Twogram"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,5192.800,5194.660,eskiden öyle yaparlarmış,c3prYnNzy3M
1,490.575,491.773,ne iftira atıyorsun bana,WCaHjQI0SKc
2,4913.528,4916.505,olmadı oğlum yanlışlık vardır,HOFjdBDqknQ
3,196.190,203.311,önemli şeyler bunlar mevlüt abi demese biz üst...,sHbEsozVweE
4,962.200,964.640,om deyin,s4MqASVecXc
...,...,...,...,...
3934202,972.540,974.040,bir nevi madalya gibi düşünün,0ocs4QH3Mcc
3934203,6475.080,6477.080,uçak barışma hediyesi,raf6rDUfiOc
3934204,3669.111,3677.132,offhand but it is around ten times what you wo...,79w04PUaRHk
3934205,7281.237,7282.194,hoş bulduk baba,8nus5IPhC5A


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram
0,adam vardı
1,al bakalım
2,al şunu
3,artık hiçbir
4,asla olmaz
...,...
125,yine geldi
126,yine yapıyorsun
127,yoksa bizi
128,yoksa sizi


In [ ]:
# Option 
# Skip if it doesn't need
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

130

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,izin ver,5710.495,5713.935,annene bir izin ver bizle bir konuşsun bize bi...,DcdoBrCvS9o
1,izin ver,148.569,153.034,seni muayene etmeme izin ver ve gidebilirsin b...,uY47FzI9kWI
2,izin ver,7864.439,7867.365,o zaman gelmemize izin ver adi pislik,uUgYlA8Afko
3,izin ver,1913.396,1915.975,lütfen yardım etmeme izin ver,xDAZpXhr_7k
4,izin ver,4670.468,4672.814,arkadaşım bak oradan yatan benim babam izin ve...,t4hQ13HMook
...,...,...,...,...,...
1275,gerçekten harika,479.416,486.264,gerçekten harika bir tarif oldu,Y8SsmYeD6IA
1276,gerçekten harika,505.150,513.880,bu görüntüde gerçekten harika ve yumurtayı güz...,nLZK95C5bkw
1277,gerçekten harika,119.255,123.540,ama uygun fiyatlılarda da bana sorarsanız mayb...,FsRYEY5PD7Y
1278,gerçekten harika,63.250,67.042,ancak kahramanlık modunda gerçekten harika ödü...,687u7xtaTkY


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,izin ver,5710.959865,5711.424730,annene bir izin ver bizle bir konuşsun bize bi...,DcdoBrCvS9o
1,izin ver,150.111455,150.923273,seni muayene etmeme izin ver ve gidebilirsin b...,uY47FzI9kWI
2,izin ver,7865.783378,7866.574189,o zaman gelmemize izin ver adi pislik,uUgYlA8Afko
3,izin ver,1915.174621,1915.975000,lütfen yardım etmeme izin ver,xDAZpXhr_7k
4,izin ver,4672.088873,4672.515418,arkadaşım bak oradan yatan benim babam izin ve...,t4hQ13HMook
...,...,...,...,...,...
1275,gerçekten harika,479.416000,483.171355,gerçekten harika bir tarif oldu,Y8SsmYeD6IA
1276,gerçekten harika,506.208182,507.795455,bu görüntüde gerçekten harika ve yumurtayı güz...,nLZK95C5bkw
1277,gerçekten harika,122.593961,123.540000,ama uygun fiyatlılarda da bana sorarsanız mayb...,FsRYEY5PD7Y
1278,gerçekten harika,64.531081,65.453459,ancak kahramanlık modunda gerçekten harika ödü...,687u7xtaTkY


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,izin ver,5710.359865,5712.024730,annene bir izin ver bizle bir konuşsun bize bi...,DcdoBrCvS9o
1,izin ver,149.511455,151.523273,seni muayene etmeme izin ver ve gidebilirsin b...,uY47FzI9kWI
2,izin ver,7865.183378,7867.174189,o zaman gelmemize izin ver adi pislik,uUgYlA8Afko
3,izin ver,1914.574621,1916.575000,lütfen yardım etmeme izin ver,xDAZpXhr_7k
4,izin ver,4671.488873,4673.115418,arkadaşım bak oradan yatan benim babam izin ve...,t4hQ13HMook
...,...,...,...,...,...
1275,gerçekten harika,478.816000,483.771355,gerçekten harika bir tarif oldu,Y8SsmYeD6IA
1276,gerçekten harika,505.608182,508.395455,bu görüntüde gerçekten harika ve yumurtayı güz...,nLZK95C5bkw
1277,gerçekten harika,121.993961,124.140000,ama uygun fiyatlılarda da bana sorarsanız mayb...,FsRYEY5PD7Y
1278,gerçekten harika,63.931081,66.053459,ancak kahramanlık modunda gerçekten harika ödü...,687u7xtaTkY


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,izin ver,5710,5712,annene bir izin ver bizle bir konuşsun bize bi...,DcdoBrCvS9o
1,izin ver,150,152,seni muayene etmeme izin ver ve gidebilirsin b...,uY47FzI9kWI
2,izin ver,7865,7867,o zaman gelmemize izin ver adi pislik,uUgYlA8Afko
3,izin ver,1915,1917,lütfen yardım etmeme izin ver,xDAZpXhr_7k
4,izin ver,4671,4673,arkadaşım bak oradan yatan benim babam izin ve...,t4hQ13HMook
...,...,...,...,...,...
1275,gerçekten harika,479,484,gerçekten harika bir tarif oldu,Y8SsmYeD6IA
1276,gerçekten harika,506,508,bu görüntüde gerçekten harika ve yumurtayı güz...,nLZK95C5bkw
1277,gerçekten harika,122,124,ama uygun fiyatlılarda da bana sorarsanız mayb...,FsRYEY5PD7Y
1278,gerçekten harika,64,66,ancak kahramanlık modunda gerçekten harika ödü...,687u7xtaTkY


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,izin ver,5710,5712,annene bir izin ver bizle bir konuşsun bize bi...,DcdoBrCvS9o,https://www.youtube.com/watch?v=DcdoBrCvS9o&t=...
1,izin ver,150,152,seni muayene etmeme izin ver ve gidebilirsin b...,uY47FzI9kWI,https://www.youtube.com/watch?v=uY47FzI9kWI&t=...
2,izin ver,7865,7867,o zaman gelmemize izin ver adi pislik,uUgYlA8Afko,https://www.youtube.com/watch?v=uUgYlA8Afko&t=...
3,izin ver,1915,1917,lütfen yardım etmeme izin ver,xDAZpXhr_7k,https://www.youtube.com/watch?v=xDAZpXhr_7k&t=...
4,izin ver,4671,4673,arkadaşım bak oradan yatan benim babam izin ve...,t4hQ13HMook,https://www.youtube.com/watch?v=t4hQ13HMook&t=...
...,...,...,...,...,...,...
1275,gerçekten harika,479,484,gerçekten harika bir tarif oldu,Y8SsmYeD6IA,https://www.youtube.com/watch?v=Y8SsmYeD6IA&t=...
1276,gerçekten harika,506,508,bu görüntüde gerçekten harika ve yumurtayı güz...,nLZK95C5bkw,https://www.youtube.com/watch?v=nLZK95C5bkw&t=...
1277,gerçekten harika,122,124,ama uygun fiyatlılarda da bana sorarsanız mayb...,FsRYEY5PD7Y,https://www.youtube.com/watch?v=FsRYEY5PD7Y&t=...
1278,gerçekten harika,64,66,ancak kahramanlık modunda gerçekten harika ödü...,687u7xtaTkY,https://www.youtube.com/watch?v=687u7xtaTkY&t=64s


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

130

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

set()

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,birlikte,3
60,buna,2
69,bunun,2
68,burada,2
67,buraya,2
...,...,...
116,şey,1
115,aslında,1
114,asla,1
113,şimdi,1


In [ ]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [25]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [26]:
set_word.difference(set_count)

{'ama',
 'anne',
 'bak',
 'bana',
 'belki',
 'bence',
 'beni',
 'benim',
 'biz',
 'bunu',
 'daha',
 'dostum',
 'dur',
 'evet',
 'gibi',
 'hakkında',
 'hayır',
 'hey',
 'iyi',
 'için',
 'lütfen',
 'merhaba',
 'nasıl',
 'o',
 'oh',
 'ol',
 'onu',
 'onun',
 'para',
 'pekala',
 'seni',
 'senin',
 'sorun',
 'teşekkürler',
 'yani',
 'yok',
 'yüzden',
 'çok',
 'öyle'}

In [27]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_30_The_Most_File_28_Word_Twogram_10_Youtube_0.6s_Timeshift_Result.xlsx',
 'Turkish_36_The_Most_File_28-172_Word_Twogram_10_Youtube_0.6s_Timeshift_Result.xlsx',
 'Turkish_130_The_Most_File_172_Word_Twogram_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [29]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [30]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 